# Общие сведения об NLP
До этого мы построили с помощью **Scikit-Learn** модели машинного обучения, например, [Классификация текста - Анализ эмоциональной окраски](http://localhost:8888/notebooks/Documents/Course%20Data%20Science/1_Applied_ML_Course/Chapter%204/7_1_Sentiment%20Analysis.ipynb). Но сегодня классификация текстов выполняется с помощью *нейронных сетей*, т.е. с использованием модели глубокого обучения.

**NLP** включает в себя различные виды задач:
- классификацию текстов
- распознавание именованных сущностей
- извлечение ключевых слов
- ответы на вопросы 
- перевод на другой язык и т.д. 

В последние годы точность таких моделей повысилась благодаря плотным векторным представлениям, трансформеру (transfonner), позволяющая выделять наиболее значимые слова и даже различать разные значения одного и того же слова.

Одним из общих элементов практически всех обрабатывающих текст нейронных сетей является слой **Embedding**. 

## Подготовка текста

Класс CountVectorizer от Scikit-Learn, преобразует строки текста в строки количества слов, преобразует символы в строчные, удаляет цифры и знаки препинания и, удаляет стопслова - такие распространенные слова, как and и the, которые, скорее всего, не
окажут большого влияния на результат.

В нейросети с глубоким обучением текст также должен быть очищен и векторизован перед использованием в обучении нейронной сети, но векторизация выполняется по-другому. Вместо того чтобы создавать таблицу количества слов, создается таблица последовательностей, содержащих представляющие отдельные слова токены. Токены часто являются индексами словаря, или лексикона, созданного на основе совокупности слов в наборе данных. Для этого в Keras имеется класс Tokenizer, которую можно рассматривать как эквивалент CountVectorizer

Вот пример, в котором Tokenizer используется для создания последовательностей из четырех строк текста:

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
lines = [
'The quick brown fox',
'Jumps over $$$ the lazy brown dog',
    'Who jumps high into the blue sky after counting 123',
'And quickly returns to earth'
]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

Метод *fit_on_texts()* создает словарь, содержащий все слова входного текста. Метод *texts_to_sequences* возвращает список
последовательностей, являющихся просто массивами индексов в словаре:

In [6]:
word_index = tokenizer.word_index
word_index

{'the': 1,
 'brown': 2,
 'jumps': 3,
 'quick': 4,
 'fox': 5,
 'over': 6,
 'lazy': 7,
 'dog': 8,
 'who': 9,
 'high': 10,
 'into': 11,
 'blue': 12,
 'sky': 13,
 'after': 14,
 'counting': 15,
 '123': 16,
 'and': 17,
 'quickly': 18,
 'returns': 19,
 'to': 20,
 'earth': 21}

In [4]:
print(sequences)

[[1, 4, 2, 5], [3, 6, 1, 7, 2, 8], [9, 3, 10, 11, 1, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21]]


Слово brown встречается в строках 1 и 2 и представлено индексом 2. Следовательно, 2 встречается в обеих последовательностях. Аналогично, индекс 3, обозначающий слово jumps, встречается в последовательностях 2 и 3. Индекс О не используется для обозначения слов; он зарезервирован. 

Можно использовать метод sequences_to_texts класса Tokenizer для обратного преобразования последовательностей в текст:

In [8]:
text = tokenizer.sequences_to_texts(sequences)
print(text)

['the quick brown fox', 'jumps over the lazy brown dog', 'who jumps high into the blue sky after counting 123', 'and quickly returns to earth']


Как видно Tokenizer преобразует текст в строчные буквы и удаляет символы, но не удаляет стоп-слова или числа. Если нужно удалить стоп-слова, то можно использовать отдельную библиотеку, например NLTK. Также можно удалить слова, содержащие цифры:

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
lines = [
'The quick brown fox',
'Jumps over $$$ the lazy brown dog',
'Who jumps high into the blue sky after counting 123',
'And quickly returns to earth'
]
def remove_stop_words(text):
    text = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)
lines = list(map(remove_stop_words, lines))
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
tokenizer.texts_to_sequences(lines)

[[3, 1, 4], [2, 5, 1, 6], [2, 7, 8, 9, 10], [11, 12, 13]]

Преобразуем теперь в текст:

In [10]:
text = tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(lines))
print(text)

['quick brown fox', 'jumps lazy brown dog', 'jumps high blue sky counting', 'quickly returns earth']


Как видите длина последовательностей первой строки - 3, второй строки - 4, третьей - 5 и последней - 3. Т.е. длина последовательностей варьируется от 3 до 5, но наша нейронная сеть ожидает, что все последовательности будут *одинаковой длины*. Поэтому с использованием функции Keras pad_sequences, обрезаем последовательности, длина которых больше указанной, и заполняя нулями последовательности, длина которых меньше указанной:

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences, maxlen=4)
padded_sequences

array([[ 1,  4,  2,  5],
       [ 1,  7,  2,  8],
       [13, 14, 15, 16],
       [18, 19, 20, 21]])

После того как текст обработан и преобразован в **последовательности**, он готов для обучения нейронной сети. 

Теперь задача слоя Embedding состоит в этом:
- преобразовать заполненные последовательности словарных токенов в массивы векторов слов, представляющих каждое слово не одним целым числом, а массивом (вектором) чисел с плавающей точкой. 

Каждое слово во входном тексте представлено вектором в слое Embedding, и по мере обучения сети векторы, представляющие отдельные слова, корректируются, чтобы можно было отразить их связь друг с другом. 

Например, в модели анализа настроений и такие слова, как "превосходный" и "потрясающий", имеют схожий смысл, поэтому векторы, представляющие эти слова в пространстве Embedding, должны быть расположены относительно близко друг к другу, чтобы фразы типа "превосходное обслуживание" и "потрясающее обслуживание" оценивались одинаково.

Реализация слоя Embedding вручную - сложная задача, поэтому Keras предлагает класс Embedding.

In [ ]:
Embedding(input_dim=10000, output_dim=32, input_length=100)

В функцию Embedding передаются три параметра по следующему порядку:

♦ размер словаря, или количество слов в словаре, созданном Tokenizer;

♦ количество измерений m в эмбеддинговом пространстве;

♦ длина n каждой заполненной последовательности.

Вы выбираете число измерений m, и каждое слово кодируется в эмбеддинговом пространстве как m-мерный вектор. Большее число измерений обеспечивает большую мощность подгонки, но также увеличивает время обучения. На практике m обычно представляет собой число от 32 до 512.

Векторы, представляющие отдельные слова в эмбеддинговом слое, изучаются в процессе обучения, так же как изучаются веса, соединяющие нейроны в соседних плотных слоях. Если количество обучающих образцов достаточно велико, при обучении сети обычно создаются эффективные векторные представления всех слов. Однако если у вас всего несколько сотен обучающих образцов, эмбединговый слой может не обладать информацией в достаточном количестве для правильной векторизации текста.

В этом случае вы можете инициализировать эмбеддинговый слой предварительно обученными эмбеддингами слов, а не полагаться на то, что он сам освоит эмбеддинги слов. В открытом доступе существует несколько популярных предварительно обученных эмбеддингов слов, например, разработанные в Стэнфорде векторы слов GloVe и разработку Word2Vec от Google. 

Пример применения предварительно обученных эмбеддингов приведен в статье "Использование предварительно обученных эмбеддингов
слов" ["Using Pretrained Word Embeddings"](https://keras.io/examples/nlp/pretrained_word_embeddings/), написанной автором Keras.

Теперь на примере классификации текста посмотрим это.

Построим архитектуру нейронной сети, выполняющей классификацию текста. Токенизированные текстовые последовательности поступают на вход эмбеддингового слоя. Выходом эмбеддингового слоя является двумерная матрица значений с плавающей точкой размером m на n, где m - количество измерений в эмбеддинговом пространстве, а n - длина последовательности. Слой Flatten, следующий за эмбеддинговым слоем, "разглаживает" двумерный выход в одномерный массив, пригодный для ввода в плотный слой, а плотный слой классифицирует значения, полученные из слоя Flatten. 

<img src="1.png" align="left"/>

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
model = Sequential()
model.add(Embedding(10000, 32, input_length=100))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Теперь давайте посмотрим его применение к задаче [фильтрации спама](http://localhost:8888/notebooks/Documents/Course%20Data%20Science/1_Applied_ML_Course/Chapter%2013/Spam%20Detection.ipynb). До этого с помощью Scikit построили модель машинного обучения, отделяющую спам от нормальных писем. Теперь построим эквивалентную модель глубокого обучения с помощью Keras и TensorFlow. Будем использовать тот же набор данных, что и раньше: набор, содержащий 1000 писем, половина из которых является спамом (обозначается единицами в столбце меток), а половина- нет (обозначается нулями в столбце меток).


**Д/З** Написать комментарий к каждой строке в задаче "Hugging Face задача.ipynb"